<a href="https://www.kaggle.com/code/bigswipp/mayo-image-segmentation?scriptVersionId=104119229" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [157]:
!pip install imutils

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import pandas as pd
import cv2 as cv
from imutils import paths
import skimage
from skimage.filters import sobel
from skimage import segmentation
from skimage.color import label2rgb
from skimage.color import rgb2hed, hed2rgb
from skimage.exposure import rescale_intensity
from skimage.measure import regionprops, regionprops_table
from scipy import ndimage as ndi
import imageio.v2 as imageio
from sklearn.preprocessing import StandardScaler

!pip install imutils

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import pandas as pd
import cv2 as cv
from imutils import paths
import skimage
from skimage.filters import sobel
from skimage import segmentation
from skimage.color import label2rgb
from skimage.color import rgb2hed, hed2rgb
from skimage.exposure import rescale_intensity
from skimage.measure import regionprops, regionprops_table
from scipy import ndimage as ndi
import imageio.v2 as imageio
from sklearn.preprocessing import StandardScaler

In [158]:
CE_folder_path = r'../input/mayo-compressed/MAYO/CE'
LAA_folder_path = r'../input/mayo-compressed/MAYO/LAA'

In [159]:
CE_images = sorted(list(paths.list_images(CE_folder_path)))
print('There are ' + str(len(CE_images)) + " images in the CE folder")
CE_images[:5]

There are 547 images in the CE folder


['../input/mayo-compressed/MAYO/CE/006388_0.png',
 '../input/mayo-compressed/MAYO/CE/008e5c_0.png',
 '../input/mayo-compressed/MAYO/CE/026c97_0.png',
 '../input/mayo-compressed/MAYO/CE/029c68_0.png',
 '../input/mayo-compressed/MAYO/CE/032f10_0.png']

In [160]:
LAA_images = sorted(list(paths.list_images(LAA_folder_path)))
print('There are ' + str(len(LAA_images)) + " images in the LAA folder")
LAA_images[:5]

There are 206 images in the LAA folder


['../input/mayo-compressed/MAYO/LAA/00c058_0.png',
 '../input/mayo-compressed/MAYO/LAA/01adc5_0.png',
 '../input/mayo-compressed/MAYO/LAA/028989_0.png',
 '../input/mayo-compressed/MAYO/LAA/03d1ec_0.png',
 '../input/mayo-compressed/MAYO/LAA/055f6a_0.png']

In [161]:
total_paths = CE_images + LAA_images
print(len(total_paths))

753


**Opening an Image**

In [172]:
def read_png(path):
    image = imageio.imread(path)
    filename = path.split('/')[-1].rstrip('.png')
    disease_class = path.split('/')[-2].rstrip('.png')
    #print("image_id: " + filename)
    return image, filename, disease_class

In [163]:
image, filename, disease_class = read_png(CE_images[16])
image.shape

image_id: 05a1ec_0


(1024, 642, 3)

**Converting the Image to Grayscale**

In [164]:
def convert_image_grayscale(image):
    gray_image = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    return gray_image

In [165]:
gray_img = convert_image_grayscale(image)

**Image Segmentation**

In [166]:
def segment_images(gray_img):
    elevation_map = sobel(gray_img) # creates image (array) that emphasizes edges
    markers = np.zeros_like(gray_img) # array of zeros in the same shape as the resized_gray_img array
    markers[gray_img >= gray_img.mean()] = 1
    markers[gray_img < gray_img.mean()] = 2
    segmented_img = segmentation.watershed(elevation_map, markers) # distinguishes objects in image 
    filled_segments = ndi.binary_fill_holes(segmented_img - 1) # method works with 0s and 1s (hence the -1, now 1 = area to keep). If 0 is eclosed by 1s -> 0 converted to 1
    labeled_segments, _ = ndi.label(filled_segments) # labels features in array, where non-zero values in input = features, and zero values = background.
    return labeled_segments

**Object Coordinates**

In [167]:
def get_object_coordinates(labeled_segments):
    unique_filenames = []
    properties =['area','bbox','convex_area','bbox_area', 'major_axis_length', 'minor_axis_length', 'eccentricity']
    df = pd.DataFrame(regionprops_table(labeled_segments, properties=properties))
    standard_scaler = StandardScaler()
    scaled_area = standard_scaler.fit_transform(df.area.values.reshape(-1,1))
    df['scaled_area'] = scaled_area
    df.sort_values(by="scaled_area", ascending=False, inplace=True)
    
    df['class'] = [disease_class] * len(scaled_area)
    df['parent_file_name'] = [filename] * len(scaled_area)
    
    for i in range(len(scaled_area)):
        unique_filenames.append(str(filename)+"_"+str(i+1))
    
    df['unique_patch_name'] = unique_filenames
    objects = df[df['scaled_area']>=.10] # tunable -> the lower it is, the more objects in the image will get boxed (original value = 0.75)
    display(objects)
    object_coordinates = [(row['bbox-0'],row['bbox-1'],row['bbox-2'],row['bbox-3'] )for index, row in objects.iterrows()]
    return objects

In [168]:
object_coordinates = get_object_coordinates(labeled_segments)

,area,bbox-0,bbox-1,bbox-2,bbox-3,convex_area,bbox_area,major_axis_length,minor_axis_length,eccentricity,scaled_area,class,parent_file_name,unique_patch_name
43,17477,547,118,689,336,22292,30956,210.461082,131.062741,0.782428,4.685262,CE,05a1ec_0,05a1ec_0_1
0,16568,16,15,160,228,21667,30672,207.670271,130.599282,0.777504,4.414905,CE,05a1ec_0,05a1ec_0_2
26,7759,257,266,373,379,9783,13108,116.919762,106.307572,0.416283,1.794911,CE,05a1ec_0,05a1ec_0_3
65,7669,794,365,904,478,9364,12430,114.846522,102.432158,0.452223,1.768143,CE,05a1ec_0,05a1ec_0_4
52,6929,660,513,763,635,9607,12566,127.606310,98.737749,0.633468,1.548051,CE,05a1ec_0,05a1ec_0_5
62,6774,768,256,863,364,7035,10260,114.584238,76.211896,0.746739,1.501950,CE,05a1ec_0,05a1ec_0_6
24,6773,234,156,333,264,7047,10692,114.107574,76.616881,0.741055,1.501653,CE,05a1ec_0,05a1ec_0_7
10,6743,118,408,220,529,9617,12342,129.614469,99.350558,0.642234,1.492730,CE,05a1ec_0,05a1ec_0_8
12,5720,142,217,258,306,6502,10324,116.134253,67.819604,0.811771,1.188467,CE,05a1ec_0,05a1ec_0_9
54,5665,678,322,795,411,6498,10413,115.774716,67.634820,0.811615,1.172109,CE,05a1ec_0,05a1ec_0_10


**Segment table**

In [173]:
def get_dataframe_from_image(path):
    image, file, disease_class = read_png(path)
    gray_img = convert_image_grayscale(image)
    labeled_segments = segment_images(gray_img)
    df = get_object_coordinates(labeled_segments)
    #return df

get_dataframe_from_image(CE_images[16])

,area,bbox-0,bbox-1,bbox-2,bbox-3,convex_area,bbox_area,major_axis_length,minor_axis_length,eccentricity,scaled_area,class,parent_file_name,unique_patch_name
43,17477,547,118,689,336,22292,30956,210.461082,131.062741,0.782428,4.685262,CE,05a1ec_0,05a1ec_0_1
0,16568,16,15,160,228,21667,30672,207.670271,130.599282,0.777504,4.414905,CE,05a1ec_0,05a1ec_0_2
26,7759,257,266,373,379,9783,13108,116.919762,106.307572,0.416283,1.794911,CE,05a1ec_0,05a1ec_0_3
65,7669,794,365,904,478,9364,12430,114.846522,102.432158,0.452223,1.768143,CE,05a1ec_0,05a1ec_0_4
52,6929,660,513,763,635,9607,12566,127.606310,98.737749,0.633468,1.548051,CE,05a1ec_0,05a1ec_0_5
62,6774,768,256,863,364,7035,10260,114.584238,76.211896,0.746739,1.501950,CE,05a1ec_0,05a1ec_0_6
24,6773,234,156,333,264,7047,10692,114.107574,76.616881,0.741055,1.501653,CE,05a1ec_0,05a1ec_0_7
10,6743,118,408,220,529,9617,12342,129.614469,99.350558,0.642234,1.492730,CE,05a1ec_0,05a1ec_0_8
12,5720,142,217,258,306,6502,10324,116.134253,67.819604,0.811771,1.188467,CE,05a1ec_0,05a1ec_0_9
54,5665,678,322,795,411,6498,10413,115.774716,67.634820,0.811615,1.172109,CE,05a1ec_0,05a1ec_0_10


**Segment table of every image**

In [170]:
def patches_df(paths_list):
    
    df_list = []
    
    for path in paths_list:
        
        image, filename, disease_class = read_png(path)
        gray_img = convert_image_grayscale(image)
        labeled_segments = segment_images(gray_img)
        object_coordinates = get_object_coordinates(labeled_segments) # df
        df_list.append(object_coordinates)
    
    combined_df = pd.concat(df_list)
    
    return combined_df

process_image(total_paths)

OSError: Cannot understand given URI: ['../input/mayo-compressed/MAYO/CE/006388_0.png', '../inp....